In [ ]:
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
import numpy as np
import fitsio
import esutil
import glob
import tqdm
import joblib

In [ ]:
import random

fnames = glob.glob("outputs/DES*.fits")
random.shuffle(fnames)

n_tiles = 100

jobs = []
for fname in tqdm.tqdm(fnames[0:n_tiles]):
    jobs.append(joblib.delayed(fitsio.read)(fname))
with joblib.Parallel(n_jobs=4, backend='loky', verbose=100) as exc:
    ds = exc(jobs)

d = esutil.numpy_util.combine_arrlist(ds)

In [ ]:
msk = (
    (d["flags"] == 0) 
    & (d["mdet_flags"] == 0) 
    & (d["mdet_s2n"] > 5) 
    & (d["mdet_T_ratio"] > 1.2)
    & (d["mfrac"] < 0.1)
)

In [ ]:
gmsk = (
    (np.sqrt(np.prod(d["mdet_g"]**2, axis=1)) < 1)
    & np.all(np.abs(d["mdet_g"]) < 1, axis=1)
)

print("frac passes cuts                :", np.mean(msk))
print("total passes cuts               :", np.sum(msk))
print("total passes cuts and shear cuts:", np.sum(msk & gmsk))
print("frac diff:", 1.0 - np.sum(msk & gmsk) / np.sum(msk))

In [ ]:
pmsk = (
    (d["flags"] == 0) 
    & (d["mdet_flags"] == 0) 
    & (d["mdet_s2n"] > 1) 
    & (d["mdet_T_ratio"] > 0.5)
    & (d["mfrac"] < 0.1)
)

plt.figure()
plt.hexbin(
    np.log10(d["mdet_s2n"][pmsk]), 
    d["mdet_T_ratio"][pmsk], 
    bins='log',
    gridsize=1000,
    extent=(0.5, 4, 0.5, 4),
)
ax = plt.gca()
ax.set_xlim(0.5, 4)
ax.set_ylim(0.5, 4)

ax.set_xlabel("log10[S/N]")
ax.set_ylabel("Tgal/Tpsf")

In [ ]:
pmsk = (
    (d["flags"] == 0) 
    & (d["mdet_flags"] == 0) 
    & (d["mdet_s2n"] > np.power(10, 3.5)) 
    & (d["mdet_T_ratio"] > 0.5)
    & (d["mfrac"] < 0.1)
)

plt.figure(figsize=(8, 6))
plt.hist(d["mdet_T_ratio"][pmsk], bins=200)
ax = plt.gca()
ax.set_xlabel("Tgal/Tpsf")
ax.vlines(1, 0, 6000, color='k')
ax.set_ylim(0, 6000)